In [ ]:
%%capture
!pip install transformers
!pip install transformers[torch]
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install nltk

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
#XSum
train_data_xsum = load_dataset("xsum", split="train[:20%]")
test_data_xsum = load_dataset("xsum", split="test[:20%]")

#CNN
train_data_cnn = load_dataset("cnn_dailymail", "3.0.0", split="train[:20%]")
test_data_cnn = load_dataset("cnn_dailymail", "3.0.0", split="test[:20%]")


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
#Untuk split train
train_data_cnn = train_data_cnn.rename_column("article", "document")
train_data_cnn = train_data_cnn.rename_column("highlights", "summary")

#Untuk split test
test_data_cnn = test_data_cnn.rename_column("article", "document")
test_data_cnn = test_data_cnn.rename_column("highlights", "summary")

In [ ]:
from datasets import concatenate_datasets

train_data = concatenate_datasets([train_data_xsum, train_data_cnn])
test_data = concatenate_datasets([test_data_xsum, test_data_cnn])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/98232 [00:00<?, ? examples/s]

Map:   0%|          | 0/4565 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="T5-Small_AbsSumm_XSumCNN",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.178900,2.262195,0.250100,0.077700,0.196000,0.196000,18.862400
2,2.149500,2.243245,0.254600,0.079300,0.199300,0.199300,18.868800
3,2.128700,2.236404,0.255500,0.079500,0.199400,0.199400,18.874700
4,2.148500,2.232910,0.256600,0.080400,0.200600,0.200700,18.890300


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=24560, training_loss=2.1661259269093067, metrics={'train_runtime': 21500.6172, 'train_samples_per_second': 18.275, 'train_steps_per_second': 1.142, 'total_flos': 1.0627518714976666e+17, 'train_loss': 2.1661259269093067, 'epoch': 4.0})

In [ ]:
from transformers import pipeline
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
summarizer = pipeline("summarization", model="DaviadiAF/T5-Small_AbsSumm_XSumCNN")
summarizer(text)

Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': "No one making under $400,000 per year will pay a penny more in taxes than any other tax payer in the U.S. it's the most aggressive action on the climate crisis in America."}]